## Neural Networks

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import scikeras
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report

In [2]:
df=pd.read_csv("Alphabets_data.csv")

In [3]:
df.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [4]:
num_samples,num_features=df.shape
num_classes=df['letter'].nunique()
class_distribution=df['letter'].value_counts()

print("Number of samples:",num_samples)
print("Number of features (excluding target):",num_features-1)
print("Number of classes:",num_classes)
print("Sample class distribution:\n",class_distribution.head())

Number of samples: 20000
Number of features (excluding target): 16
Number of classes: 26
Sample class distribution:
 letter
U    813
D    805
P    803
T    796
M    792
Name: count, dtype: int64


In [5]:
print("Missing values:",df.isnull().sum().sum())

Missing values: 0


In [6]:
X=df.drop(columns=['letter'])
y=df['letter']
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X)
print("Scaled data shape:",X_scaled.shape)

Scaled data shape: (20000, 16)


In [7]:
encoder=LabelEncoder()
y_encoded=encoder.fit_transform(y)

In [8]:
# Train-test split
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y_encoded,test_size=0.2,random_state=42)

In [9]:
#ANN model- before tuning
model1=Sequential()
model1.add(Dense(64,input_dim=X_train.shape[1],activation='relu'))
model1.add(Dense(32,activation='relu'))
model1.add(Dense(len(encoder.classes_),activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

c:\Users\Srees_lappy\Desktop\dscodes\EXCELR\venvex\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
#Train
history=model1.fit(X_train,y_train,epochs=20,batch_size=32,validation_split=0.2,verbose=1)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2335 - loss: 2.7710 - val_accuracy: 0.4109 - val_loss: 2.1321
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4894 - loss: 1.8151 - val_accuracy: 0.5206 - val_loss: 1.6526
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5759 - loss: 1.5022 - val_accuracy: 0.5975 - val_loss: 1.4256
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6209 - loss: 1.3441 - val_accuracy: 0.6416 - val_loss: 1.3197
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6504 - loss: 1.2387 - val_accuracy: 0.6584 - val_loss: 1.2311
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6745 - loss: 1.1653 - val_accuracy: 0.6816 - val_loss: 1.1557
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6952 - loss: 1.1033 - val_accuracy: 0.7094 - val_loss: 1.1015
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7077 - loss: 1.0539 - val_accuracy: 0.

In [11]:
#Evaluate
loss,acc=model1.evaluate(X_test,y_test,verbose=0)
print("Model1 (Default ANN) Test Accuracy:",acc)

Model1 (Default ANN) Test Accuracy: 0.7827500104904175


In [12]:
#Build model function for tuning
def create_model(neurons=32,layers=1,activation='relu',lr=0.001):
    model=Sequential()
    model.add(Dense(neurons,input_dim=X_train.shape[1],activation=activation))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation=activation))
    model.add(Dense(len(encoder.classes_),activation='softmax'))
    opt=Adam(learning_rate=lr)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    return model

In [13]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Wrap model for sklearn (Tuned ANN)
tuned_model=KerasClassifier(model=create_model,verbose=0)

#Hyperparameter space
param_dist={
    "model__neurons":[32,64,128],
    "model__layers":[1,2,3],
    "model__activation":['relu','tanh'],
    "model__lr":[0.001,0.01],
    "epochs":[10,20],
    "batch_size":[32,64]
}

In [15]:
#Randomized search
search=RandomizedSearchCV(estimator=tuned_model,param_distributions=param_dist,n_iter=5,cv=3,verbose=1,n_jobs=-1)
search_result=search.fit(X_train,y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\Srees_lappy\Desktop\dscodes\EXCELR\venvex\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
#Best results
print("Best Params:",search_result.best_params_)
print("Best CV Score:",search_result.best_score_)

Best Params: {'model__neurons': 64, 'model__lr': 0.01, 'model__layers': 2, 'model__activation': 'tanh', 'epochs': 10, 'batch_size': 64}
Best CV Score: 0.8638764168167334


In [17]:
#Evaluate on test set
test_acc=search_result.score(X_test,y_test)
print("Test Accuracy with Best Params:",test_acc)

Test Accuracy with Best Params: 0.87075


In [18]:
y_pred_default=model1.predict(X_test)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:
y_pred_default=y_pred_default.argmax(axis=1)  # convert probabilities to class labels

In [20]:
#Predictions with tuned model
y_pred_tuned=search_result.predict(X_test)

In [21]:
#Evaluation
print("\nFirst Model Performance")
print("Accuracy:",accuracy_score(y_test,y_pred_default))
print("Precision:",precision_score(y_test,y_pred_default,average='macro'))
print("Recall:",recall_score(y_test,y_pred_default,average='macro'))
print("F1-score:",f1_score(y_test,y_pred_default,average='macro'))


First Model Performance
Accuracy: 0.78275
Precision: 0.7846715974634135
Recall: 0.7809263649714265
F1-score: 0.7792361074046247


In [22]:
print("Tuned Model Performance")
print("Accuracy:",accuracy_score(y_test,y_pred_tuned))
print("Precision:",precision_score(y_test,y_pred_tuned,average='macro'))
print("Recall:",recall_score(y_test,y_pred_tuned,average='macro'))
print("F1-score:",f1_score(y_test,y_pred_tuned,average='macro'))

Tuned Model Performance
Accuracy: 0.87075
Precision: 0.8786099379421834
Recall: 0.8712105607616886
F1-score: 0.8699496714561913


In [23]:
print("Classification Report (Tuned Model):",classification_report(y_test,y_pred_tuned))

Classification Report (Tuned Model):               precision    recall  f1-score   support

           0       0.91      0.94      0.92       149
           1       0.85      0.80      0.82       153
           2       0.89      0.82      0.86       137
           3       0.84      0.83      0.84       156
           4       0.81      0.85      0.83       141
           5       0.74      0.89      0.81       140
           6       0.95      0.63      0.76       160
           7       0.72      0.71      0.71       144
           8       1.00      0.79      0.89       146
           9       0.86      0.91      0.89       149
          10       0.61      0.94      0.74       130
          11       0.97      0.90      0.93       155
          12       0.93      0.96      0.94       168
          13       1.00      0.87      0.93       151
          14       0.90      0.88      0.89       145
          15       0.98      0.83      0.90       173
          16       0.93      0.86      0.90 

#### Discussion

* The baseline ANN (Model1) achieved reasonable accuracy, but its performance across precision, recall, and F1-score showed room for improvement. 
* After applying hyperparameter tuning with RandomizedSearchCV, the tuned ANN achieved higher accuracy and better balanced metrics, indicating improved generalization. 
* This improvement highlights the importance of hyperparameters such as the number of layers, neurons, activation functions, and learning rate in optimizing ANN performance.
* Overall, hyperparameter tuning significantly enhanced the classification results compared to the default model.